In [4]:
!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels


In [84]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, from_utc_timestamp, when, lower, lit, trim, col, count
from pyspark.sql.types import StringType
import pandas as pd


In [6]:
spark = SparkSession.builder.appName("AzureBilling").getOrCreate()

25/03/09 08:25:31 WARN Utils: Your hostname, green-nbjupyterhub5 resolves to a loopback address: 127.0.0.1; using 10.0.0.178 instead (on interface ens5)
25/03/09 08:25:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/09 08:25:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [56]:
df_costs = spark.read.csv("focus_azure_billing/EA-Cost-Actual.csv", header=True, inferSchema=True)
df_prices = spark.read.csv("focus_azure_billing/EA-Prices.csv", header=True, inferSchema=True)

In [57]:
df_costs_pd = df_costs.toPandas()
df_prices_pd = df_prices.toPandas()

In [59]:
display(df_costs_pd)

,InvoiceSectionName,AccountName,AccountOwnerId,SubscriptionId,SubscriptionName,ResourceGroup,ResourceLocation,Date,ProductName,MeterCategory,...,BillingProfileName,InvoiceSectionId,IsAzureCreditEligible,PartNumber,PayGPrice,PlanName,ServiceFamily,CostAllocationRuleName,benefitId,benefitName
0,ACM,ACM Team,acm@testea.onmicrosoft.com,ed570627-0265-4620-bb42-bae06bcfa914,Trey Research IT,petsupply-rg,All Regions,06/12/2024,Azure DNS - Public Queries,Azure DNS,...,USD,06/01/2024,06/30/2024,8611537,Trey Research Demo (DO NOT USE),None,True,T6Z-00026,0.4,Public
1,ACM,ACM Team,acm@testea.onmicrosoft.com,64e355d7-997c-491d-b0c1-8414dccfcf42,Trey Research R&D Playground,devtestlab,EastUS2,06/12/2024,Bandwidth Inter-Region - Intra Continent Data ...,Bandwidth,...,None,None,Azure,Microsoft,Usage,UsageBased,OnDemand,None,8611537,Trey Research Demo (DO NOT USE)
2,ACM,ACM Team,acm@testea.onmicrosoft.com,9ec51cfd-5ca7-4d76-8101-dd0a4abc5674,Trey Research Corporate,MinorEnigma,northeurope,06/12/2024,Microsoft Defender for Storage - Free Transact...,Microsoft Defender for Cloud,...,Trey Research Demo (DO NOT USE),USD,06/01/2024,06/30/2024,8611537,Trey Research Demo (DO NOT USE),None,True,AAN-32388,0
3,ACM,ACM Team,acm@testea.onmicrosoft.com,64e355d7-997c-491d-b0c1-8414dccfcf42,Trey Research R&D Playground,ClancyTest,CentralUS,06/12/2024,Tables - LRS Data Stored,Storage,...,06/01/2024,06/30/2024,8611537,Trey Research Demo (DO NOT USE),None,True,N9H-00637,0.045,Standard,Compute
4,ACM,ACM Team,acm@testea.onmicrosoft.com,64e355d7-997c-491d-b0c1-8414dccfcf42,Trey Research R&D Playground,ftk-integration-tests,EastUS,06/12/2024,Files - List Operations,Storage,...,USD,06/01/2024,06/30/2024,8611537,Trey Research Demo (DO NOT USE),None,True,N9H-00817,0.015,Standard
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135248,ACM,ACM Team,acm@testea.onmicrosoft.com,64e355d7-997c-491d-b0c1-8414dccfcf42,Trey Research R&D Playground,ftk-integration-tests,EastUS,06/05/2024,Tables - Batch Write Operations,Storage,...,USD,06/01/2024,06/30/2024,8611537,Trey Research Demo (DO NOT USE),None,True,N9H-00758,0.00036,Standard
135249,ACM,ACM Team,acm@testea.onmicrosoft.com,64e355d7-997c-491d-b0c1-8414dccfcf42,Trey Research R&D Playground,ftk-integration-tests,EastUS,06/05/2024,Microsoft Defender for Storage - Standard Tran...,Microsoft Defender for Cloud,...,06/30/2024,8611537,Trey Research Demo (DO NOT USE),None,True,AAA-97331,0.02,Standard,Compute,None
135250,ACM,ACM Team,acm@testea.onmicrosoft.com,64e355d7-997c-491d-b0c1-8414dccfcf42,Trey Research R&D Playground,ftk-integration-tests,EastUS,06/05/2024,Queues v2 - LRS - Class 1 Operations - US East,Storage,...,USD,06/01/2024,06/30/2024,8611537,Trey Research Demo (DO NOT USE),None,True,AAA-71362,0.004,Standard
135251,ACM,ACM Team,acm@testea.onmicrosoft.com,64e355d7-997c-491d-b0c1-8414dccfcf42,Trey Research R&D Playground,ftk-integration-tests,EastUS,06/05/2024,Microsoft Defender for Storage - Standard Tran...,Microsoft Defender for Cloud,...,Trey Research Demo (DO NOT USE),USD,06/01/2024,06/30/2024,8611537,Trey Research Demo (DO NOT USE),None,True,AAA-97331,0.02


In [90]:
total_rows_costs = df_costs.count()
fill_rates_costs = df_costs.select(*[(count(col(c)) / total_rows_costs * 100).alias(f"{c}_fill_rate") for c in df_costs.columns])
fill_rates_costs_pd = fill_rates_costs.toPandas()

display(fill_rates_costs_pd)

,InvoiceSectionName_fill_rate,AccountName_fill_rate,AccountOwnerId_fill_rate,SubscriptionId_fill_rate,SubscriptionName_fill_rate,ResourceGroup_fill_rate,ResourceLocation_fill_rate,Date_fill_rate,ProductName_fill_rate,MeterCategory_fill_rate,...,BillingProfileName_fill_rate,InvoiceSectionId_fill_rate,IsAzureCreditEligible_fill_rate,PartNumber_fill_rate,PayGPrice_fill_rate,PlanName_fill_rate,ServiceFamily_fill_rate,CostAllocationRuleName_fill_rate,benefitId_fill_rate,benefitName_fill_rate
0,100.0,100.0,100.0,100.0,100.0,99.81664,100.0,100.0,100.0,100.0,...,80.994137,80.099517,87.703045,89.996525,86.434312,66.468766,73.097824,81.401522,87.829475,88.669383


In [60]:
display(df_prices_pd)

,EnrollmentNumber,MeterID,MeterName,MeterType,MeterCategory,MeterSubCategory,ServiceFamily,Product,SkuID,ProductID,...,EffectiveStartDate,EffectiveEndDate,UnitPrice,BasePrice,MarketPrice,CurrencyCode,IncludedQuantity,OfferID,Term,PriceType
0,8611537,bb20ba32-252b-5422-9b77-01aa1dee3d87,DC8ads v5 Low Priority,1 Compute Hour,Virtual Machines,DCadsv5-series Linux,Compute,DCadsv5-series Linux - DC8ads v5 Low Priority ...,000K,DZH318Z09B5Z,...,06/01/2024 00:00:00,07/01/2024 00:00:00,82.40,90.6000,82.40,USD,0,MS-AZR-0148P,None,Consumption
1,8611537,dfaeaca2-77a6-547e-8445-a972ab24d8d5,Commitment Tier CSTT Disconnected 100K Unit,1 Unit/Year,Cognitive Services,Speech - disconnected,AI + Machine Learning,Speech - disconnected - Commitment Tier CSTT D...,00DF,DZH318Z09WSW,...,06/01/2024 00:00:00,07/01/2024 00:00:00,547200.00,638400.0000,547200.00,USD,0,MS-AZR-0017P,None,Consumption
2,8611537,3972e35b-1884-5241-8e32-02b35bdfb82e,D4pds v5 Low Priority,1 Compute Hour,Virtual Machines,Dpdsv5 Series Windows,Compute,Virtual Machines Dpdsv5 Series Windows - D4pds...,00NB,DZH318Z092SC,...,06/01/2024 00:00:00,07/01/2024 00:00:00,0.04,0.0434,0.04,USD,0,MS-AZR-0148P,None,Consumption
3,8611537,eb59fe67-f2d7-5e4b-b22e-c146438e5142,E64ds v4 Low Priority,1 Compute Hour,Virtual Machines,Edsv4 Series Windows,Compute,Virtual Machines Edsv4 Series Windows - E64ds ...,029F,DZH318Z0CSHB,...,06/01/2024 00:00:00,07/01/2024 00:00:00,34.30,36.1100,34.30,USD,0,MS-AZR-0017P,None,Consumption
4,8611537,8c7ace14-9c5d-5139-9d6a-e8241e19d1fc,E32pds v5 Low Priority,1 Compute Hour,Virtual Machines,Epdsv5 Series Windows,Compute,Virtual Machines Epdsv5 Series Windows - E32pd...,00Q9,DZH318Z092SG,...,06/01/2024 00:00:00,07/01/2024 00:00:00,0.43,0.4330,0.43,USD,0,MS-AZR-0148P,None,Consumption
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753711,8611537,b715c324-fffe-53ab-8308-0ab74de37ec7,None,1 Compute Hour,Virtual Machines,Virtual Machines Dlsv5 Series,Compute,"Reserved VM Instance, Standard_D48ls_v5, CA Ce...",01SH,DZH318Z08M9L,...,06/19/2024 00:00:00,None,22290.00,NaN,22290.00,USD,0,None,P3Y,ReservedInstance
753712,8611537,5b41061d-0eb3-5b91-8043-3fe96ff9f655,None,1 Compute Hour,Virtual Machines,Virtual Machines Ev5 Series,Compute,"Reserved VM Instance, Standard_E16_v5, US Sout...",031P,DZH318Z08M9J,...,06/19/2024 00:00:00,None,6146.00,NaN,6146.00,USD,0,None,P1Y,ReservedInstance
753713,8611537,a14fd6cf-9c30-5a3a-917e-23317c293fba,None,1 App Service Hour,Azure App Service,Azure App Service Isolated v2 Plan,Compute,"Azure App Service Isolated v2 Plan, I2 v2, SE ...",02MW,DZH318Z0DCR5,...,06/19/2024 00:00:00,None,18317.00,NaN,18317.00,USD,0,None,P3Y,ReservedInstance
753714,8611537,262ad2b2-7956-569a-9a33-d72fa79c79f1,None,1 Compute Hour,Virtual Machines,Virtual Machines Bsv2 Series,Compute,"Reserved VM Instance, Standard_B16ls_v2, QA Ce...",03G7,DZH318Z0K9K9,...,06/19/2024 00:00:00,None,6778.00,NaN,6778.00,USD,0,None,P3Y,ReservedInstance


In [92]:
total_rows_prices = df_prices.count()
fill_rates_prices = df_prices.select(*[(count(col(c)) / total_rows_prices * 100).alias(f"{c}_fill_rate") for c in df_prices.columns])
fill_rates_prices_pd = fill_rates_prices.toPandas()

display(fill_rates_prices_pd)

,EnrollmentNumber_fill_rate,MeterID_fill_rate,MeterName_fill_rate,MeterType_fill_rate,MeterCategory_fill_rate,MeterSubCategory_fill_rate,ServiceFamily_fill_rate,Product_fill_rate,SkuID_fill_rate,ProductID_fill_rate,...,EffectiveStartDate_fill_rate,EffectiveEndDate_fill_rate,UnitPrice_fill_rate,BasePrice_fill_rate,MarketPrice_fill_rate,CurrencyCode_fill_rate,IncludedQuantity_fill_rate,OfferID_fill_rate,Term_fill_rate,PriceType_fill_rate
0,100.0,100.0,88.849514,100.0,100.0,96.04546,100.0,100.0,100.0,100.0,...,100.0,88.849514,100.0,72.769717,100.0,100.0,100.0,88.849514,27.230283,100.0


In [64]:
url = "https://raw.githubusercontent.com/finopsfoundation/focus_converters/dev/focus_converter_base/focus_converter/conversion_configs/azure/mapping_files/azure_category_mapping.csv"
service_mapping_df = pd.read_csv(url)
service_mapping_spark_df = spark.createDataFrame(service_mapping_df)


In [66]:
df_costs_transformed = df_costs.select(
        col("MeterId").alias("CostMeterId"),
        col("AvailabilityZone").alias("AvailabilityZone"),
        col("CostInBillingCurrency").alias("BilledCost"),
        col("BillingAccountId").alias("BillingAccountId"),
        col("BillingAccountName").alias("BillingAccountName"),
        col("BillingCurrencyCode").alias("BillingCurrency"),
        from_utc_timestamp(to_timestamp(col("BillingPeriodEndDate"), "MM/dd/yyyy"), "UTC").alias("BillingPeriodEnd"),
        from_utc_timestamp(to_timestamp(col("BillingPeriodStartDate"), "MM/dd/yyyy"), "UTC").alias("BillingPeriodStart"),
        col("ChargeType"),
        col("Frequency"),
        col("benefitId"),
        col("UnitOfMeasure").alias("ConsumedUnit"),
        col("PricingModel"),
        col("ProductName").alias("ChargeDescription"),
        from_utc_timestamp(to_timestamp(col("Date"), "MM/dd/yyyy"), "UTC").alias("ChargePeriodEnd"),
        from_utc_timestamp(to_timestamp(col("BillingPeriodStartDate"), "MM/dd/yyyy"), "UTC").alias("ChargePeriodStart"),
        col("benefitId").alias("CommitmentDiscountId"),
        col("benefitName").alias("CommitmentDiscountName"),
        col("Quantity").alias("ConsumedQuantity"),
        col("CostInBillingCurrency").alias("EffectiveCost"),
        col("PublisherName").alias("Publisher"),
        col("ResourceLocation").alias("RegionId"),
        col("ResourceId").alias("ResourceId"),
        col("ResourceName").alias("ResourceName"),
        col("ConsumedService"),
        col("ConsumedService").alias("ServiceName"),
        col("SubscriptionId").alias("SubAccountId"),
        col("SubscriptionName").alias("SubAccountName"),
        col("PartNumber"),
        col("OfferID")
    ).withColumn(
        "ChargeCategory",
        when(col("ChargeType").isin(["Usage", "UnusedReservation", "UnusedSavingsPlan"]), "Usage")
        .when(col("ChargeType") == "Purchase", "Purchase")
        .when(col("ChargeType") == "Tax", "Tax")
        .otherwise("Adjustment")
    ).withColumn(
        "ChargeFrequency",
        when(col("Frequency") == "OneTime", "One-Time")
        .when(col("Frequency") == "Recurring", "Recurring")
        .when(col("Frequency") == "UsageBased", "Usage-Based")
        .otherwise("Other") 
    ).withColumn(
        "PricingCategory",
        when(col("PricingModel") == "OnDemand", "On-Demand")
        .when(col("PricingModel") == "Spot", "Dynamic")
        .when(col("PricingModel").isin(["Reservation", "Savings Plans"]), "Commitment Discount")
        .otherwise("Other")
    ).withColumn(
        "CommitmentDiscountCategory",
        when(lower(col("benefitId")).contains("/microsoft.capacity/"), "Usage")
        .when(lower(col("benefitId")).contains("/microsoft.billingbenefits/"), "Spend")
        .otherwise(None)
    ).withColumn(
        "CommitmentDiscountType",
        when(lower(col("benefitId")).contains("/microsoft.capacity/"), "Reservation")
        .when(lower(col("benefitId")).contains("/microsoft.billingbenefits/"), "Savings Plan")
        .otherwise(None)
    ).withColumn(
        "InvoiceIssuer", lit("Microsoft")
    ).withColumn(
        "Provider", lit("Azure")
    ).withColumn(
        "SkuPriceId", lit(None)
    ).drop("ChargeType", "Frequency", "benefitId", "UnitOfMeasure", "PricingModel")


In [68]:
df = df_costs_transformed.join(
        service_mapping_spark_df,
        df_costs_transformed["ConsumedService"] == service_mapping_spark_df["ConsumedService"],
        "left"
    ).select(
        df_costs_transformed["*"],
        service_mapping_spark_df["ServiceCategory"]
    ).drop("ConsumedService")


In [104]:
df_pd = df.toPandas()

In [105]:
display(df_pd)

,CostMeterId,AvailabilityZone,BilledCost,BillingAccountId,BillingAccountName,BillingCurrency,BillingPeriodEnd,BillingPeriodStart,ConsumedUnit,ChargeDescription,...,OfferID,ChargeCategory,ChargeFrequency,PricingCategory,CommitmentDiscountCategory,CommitmentDiscountType,InvoiceIssuer,Provider,SkuPriceId,ServiceCategory
0,155afecf-5804-4ca4-ae3d-2cff595ab8d4,UsageBased,0.000025,OnDemand,None,8611537,NaT,NaT,1,Logic Apps - Consumption Built-in Actions,...,"""""org"""": """"trey""""",Adjustment,Other,Other,None,None,Microsoft,Azure,None,None
1,3b2bbe89-0295-459e-b5e9-7e43b90318da,UsageBased,0.021824,OnDemand,None,8611537,NaT,NaT,1 GB/Month,Logic Apps - Consumption Data Retention,...,"""""org"""": """"trey""""",Adjustment,Other,Other,None,None,Microsoft,Azure,None,None
2,3d2036fb-cad3-47cf-98cb-0a66f53be6cd,UsageBased,0.000250,OnDemand,None,8611537,NaT,NaT,1,Logic Apps - Consumption Standard Connector Ac...,...,"""""org"""": """"trey""""",Adjustment,Other,Other,None,None,Microsoft,Azure,None,None
3,155afecf-5804-4ca4-ae3d-2cff595ab8d4,UsageBased,0.000025,OnDemand,None,8611537,NaT,NaT,1,Logic Apps - Consumption Built-in Actions,...,"""""org"""": """"trey""""",Adjustment,Other,Other,None,None,Microsoft,Azure,None,None
4,3d2036fb-cad3-47cf-98cb-0a66f53be6cd,UsageBased,0.000250,OnDemand,None,8611537,NaT,NaT,1,Logic Apps - Consumption Standard Connector Ac...,...,"""""org"""": """"trey""""",Adjustment,Other,Other,None,None,Microsoft,Azure,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135248,6a11eb4c-01de-4438-88e0-e95b41869197,Microsoft,0.000000,Usage,UsageBased,OnDemand,NaT,NaT,1 GB,Insight and Analytics - Standard Data Included...,...,"""""Cost department"""": """"Marketing""""",Adjustment,Other,Other,None,None,Microsoft,Azure,None,Management and Governance
135249,6a11eb4c-01de-4438-88e0-e95b41869197,Microsoft,0.000000,Usage,UsageBased,OnDemand,NaT,NaT,1 GB,Insight and Analytics - Standard Data Included...,...,"""""Cost department"""": """"Marketing""""",Adjustment,Other,Other,None,None,Microsoft,Azure,None,Management and Governance
135250,6a11eb4c-01de-4438-88e0-e95b41869197,None,0.000000,8611537,Trey Research Demo (DO NOT USE),USD,2024-06-30,2024-06-01,1 GB,Insight and Analytics - Standard Data Included...,...,MS-AZR-0017P,Usage,Usage-Based,On-Demand,None,None,Microsoft,Azure,None,Management and Governance
135251,ffbd06c0-e72e-42b6-b612-f03144b476df,Azure,-0.475225,Microsoft,Usage,UsageBased,NaT,NaT,1 GB,Log Analytics - Pay-as-you-go Data Ingestion,...,"""""env"""": """"trey""""",Adjustment,Other,Other,None,None,Microsoft,Azure,None,Management and Governance


In [70]:
joined_df = df_costs.alias("c").join(
        df_prices.alias("p"),
        (trim(col("c.MeterId")).cast(StringType()) == trim(col("p.MeterID")).cast(StringType())) &
        (trim(col("c.BillingCurrencyCode")).cast(StringType()) == trim(col("p.CurrencyCode")).cast(StringType())) &
        (trim(col("c.PartNumber")).cast(StringType()) == trim(col("p.PartNumber")).cast(StringType())) & 
        (trim(col("c.OfferID")).cast(StringType()) == trim(col("p.OfferID")).cast(StringType())),
        "left"
    ).select(
        col("c.MeterId"),
        col("c.BillingCurrencyCode"),
        col("c.PartNumber").alias("CostPartNumber"),
        col("c.OfferId").alias("CostOfferId"),
        col("c.Quantity").alias("CostQuantity"),
        col("c.UnitPrice").alias("CostUnitPrice"),
        col("p.UnitPrice").alias("PriceUnitPrice"),
        col("p.UnitOfMeasure").alias("PriceUnitOfMeasure"),
        col("p.MeterType").alias("PriceMeterType"),
        col("p.MeterRegion").alias("PriceMeterRegion"),
        col("p.ServiceFamily").alias("PriceServiceFamily"),
        col("p.SkuID").alias("PriceSkuID")
    ).withColumn(
        "ContractedCost", col("CostQuantity") * col("CostUnitPrice")
    ).withColumn(
        "ContractedUnitPrice", col("CostUnitPrice")
    ).withColumn(
        "ListCost", col("CostQuantity") * col("PriceUnitPrice")
    ).withColumn(
        "ListUnitPrice", col("PriceUnitPrice")
    ).withColumn(
        "PricingQuantity", col("PriceUnitOfMeasure")
    ).withColumn(
        "PricingUnit", col("PriceMeterType")
    ).withColumn(
        "RegionName", col("PriceMeterRegion")
    ).withColumn(
        "ResourceType", col("PriceServiceFamily")
    ).withColumn(
        "SkuId", col("PriceSkuID")
    ).drop(
        "CostQuantity",
        "CostUnitPrice",
        "PriceUnitPrice",
        "PriceUnitOfMeasure",
        "PriceMeterType",
        "PriceMeterRegion",
        "PriceServiceFamily",
        "PriceSkuID"
    )


In [72]:
df_focus = df.join(
        joined_df.dropDuplicates(["MeterId", "BillingCurrencyCode", "CostPartNumber", "CostOfferId"]),
        (trim(col("CostMeterId")).cast(StringType()) == trim(col("MeterId")).cast(StringType())) &
        (trim(col("BillingCurrency")).cast(StringType()) == trim(col("BillingCurrencyCode")).cast(StringType())) &
        (trim(col("PartNumber")).cast(StringType()) == trim(col("CostPartNumber")).cast(StringType())) & 
        (trim(col("OfferID")).cast(StringType()) == trim(col("CostOfferId")).cast(StringType())),
        "left"
    ).select(
        "ContractedCost",
        "ContractedUnitPrice",
        "ListCost",
        "ListUnitPrice",
        "PricingQuantity",
        "PricingUnit",
        "RegionName",
        "ResourceType",
        "SkuId",
        "AvailabilityZone",
        "BilledCost",
        "BillingAccountId",
        "BillingAccountName",
        "BillingCurrency",
        "BillingPeriodEnd",
        "BillingPeriodStart",
        "ChargeCategory",
        "ChargeDescription",
        "ChargeFrequency",
        "ChargePeriodEnd",
        "ChargePeriodStart",
        "CommitmentDiscountCategory",
        "CommitmentDiscountId",
        "CommitmentDiscountName",
        "CommitmentDiscountType",
        "ConsumedQuantity",
        "ConsumedUnit",
        "EffectiveCost",
        "InvoiceIssuer",
        "PricingCategory",
        "Provider",
        "Publisher",
        "RegionId",
        "ResourceId",
        "ResourceName",
        "ServiceCategory",
        "ServiceName",
        "SkuPriceId",
        "SubAccountId",
        "SubAccountName"
    )


In [80]:
df_focus_pd = df_focus.toPandas()

In [81]:
display(df_focus_pd)

,ContractedCost,ContractedUnitPrice,ListCost,ListUnitPrice,PricingQuantity,PricingUnit,RegionName,ResourceType,SkuId,AvailabilityZone,...,Provider,Publisher,RegionId,ResourceId,ResourceName,ServiceCategory,ServiceName,SkuPriceId,SubAccountId,SubAccountName
0,NaN,None,NaN,NaN,None,None,None,None,None,UsageBased,...,Azure,None,WestUS,/SUBSCRIPTIONS/1CAAA5A3-2B66-438E-8AB4-BCE37D5...,None,None,MICROSOFT.LOGIC,None,1caaa5a3-2b66-438e-8ab4-bce37d518c5d,Cost Management Research
1,NaN,None,NaN,NaN,None,None,None,None,None,UsageBased,...,Azure,None,WestUS,/SUBSCRIPTIONS/1CAAA5A3-2B66-438E-8AB4-BCE37D5...,None,None,MICROSOFT.LOGIC,None,1caaa5a3-2b66-438e-8ab4-bce37d518c5d,Cost Management Research
2,NaN,None,NaN,NaN,None,None,None,None,None,UsageBased,...,Azure,None,WestUS,/SUBSCRIPTIONS/1CAAA5A3-2B66-438E-8AB4-BCE37D5...,None,None,MICROSOFT.LOGIC,None,1caaa5a3-2b66-438e-8ab4-bce37d518c5d,Cost Management Research
3,NaN,None,NaN,NaN,None,None,None,None,None,UsageBased,...,Azure,None,WestUS,/SUBSCRIPTIONS/1CAAA5A3-2B66-438E-8AB4-BCE37D5...,None,None,MICROSOFT.LOGIC,None,1caaa5a3-2b66-438e-8ab4-bce37d518c5d,Cost Management Research
4,NaN,None,NaN,NaN,None,None,None,None,None,UsageBased,...,Azure,None,WestUS,/SUBSCRIPTIONS/1CAAA5A3-2B66-438E-8AB4-BCE37D5...,None,None,MICROSOFT.LOGIC,None,1caaa5a3-2b66-438e-8ab4-bce37d518c5d,Cost Management Research
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135248,NaN,MS-AZR-0017P,NaN,NaN,None,None,None,None,None,None,...,Azure,kubernetes,EastUS,/subscriptions/9ec51cfd-5ca7-4d76-8101-dd0a4ab...,"""""org"""": """"trey""""",Networking,microsoft.network,None,9ec51cfd-5ca7-4d76-8101-dd0a4abc5674,Trey Research Corporate
135249,NaN,MS-AZR-0017P,NaN,NaN,None,None,None,None,None,None,...,Azure,kubernetes,EastUS,/subscriptions/9ec51cfd-5ca7-4d76-8101-dd0a4ab...,"""""org"""": """"trey""""",Networking,microsoft.network,None,ed570627-0265-4620-bb42-bae06bcfa914,Trey Research IT
135250,NaN,MS-AZR-0017P,NaN,NaN,None,None,None,None,None,None,...,Azure,kubernetes,EastUS,/subscriptions/9ec51cfd-5ca7-4d76-8101-dd0a4ab...,"""""org"""": """"trey""""",Networking,microsoft.network,None,ed570627-0265-4620-bb42-bae06bcfa914,Trey Research IT
135251,NaN,MS-AZR-0017P,NaN,NaN,None,None,None,None,None,None,...,Azure,kubernetes,EastUS,/subscriptions/9ec51cfd-5ca7-4d76-8101-dd0a4ab...,"""""org"""": """"trey""""",Networking,microsoft.network,None,9ec51cfd-5ca7-4d76-8101-dd0a4abc5674,Trey Research Corporate


In [96]:
df_focus_pd.to_csv("focus_azure_billing/focus_billing.csv", index=False)

In [98]:
total_rows = df_focus.count()
fill_rates = df_focus.select(*[(count(col(c)) / total_rows * 100).alias(f"{c}_fill_rate") for c in df_focus.columns])
fill_rates_pd = fill_rates.toPandas()

display(fill_rates_pd) 

,ContractedCost_fill_rate,ContractedUnitPrice_fill_rate,ListCost_fill_rate,ListUnitPrice_fill_rate,PricingQuantity_fill_rate,PricingUnit_fill_rate,RegionName_fill_rate,ResourceType_fill_rate,SkuId_fill_rate,AvailabilityZone_fill_rate,...,Provider_fill_rate,Publisher_fill_rate,RegionId_fill_rate,ResourceId_fill_rate,ResourceName_fill_rate,ServiceCategory_fill_rate,ServiceName_fill_rate,SkuPriceId_fill_rate,SubAccountId_fill_rate,SubAccountName_fill_rate
0,3.048361,17.116071,3.048361,3.048361,3.048361,3.048361,1.129735,3.048361,3.048361,76.625287,...,100.0,49.266929,100.0,100.0,85.842828,6.945502,100.0,0.0,100.0,100.0
